In [1]:
import dataframe_image as dfi
import matplotlib.pyplot as plt
import mlrose_hiive
import mlrose
import numpy as np
import pandas as pd
import time
import warnings

from sklearn.metrics import accuracy_score
from sklearn.model_selection import (GridSearchCV, train_test_split, validation_curve)   
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
np.random.seed(42)

In [29]:
# Fitness Functions
fitness_functions = [('fourpeaks', mlrose_hiive.FourPeaks()), ('contpeaks', mlrose_hiive.ContinuousPeaks()), ('onemax',mlrose_hiive.OneMax())]

In [54]:
# RHC tuning

best_restart_param = None
best_restart_fitness_value = None
for func in fitness_functions:
    fitness_function = func[1]
    print("Find best RHC hyperparam for {}".format(str(fitness_function)))
    for i in [0, 25, 75, 100]:
        problem = mlrose_hiive.DiscreteOpt(length=100, fitness_fn=fitness_function, maximize=True)
        rhc_best_state, rhc_best_fitness, rhc_fitness_curve = mlrose_hiive.random_hill_climb(problem, 
                                                                                     max_attempts = 500, 
                                                                                     max_iters=500, 
                                                                                     curve=True, 
                                                                                     random_state=42,
                                                                                     restarts = i)
        if not best_restart_fitness_value:
            best_restart_param = i
            best_restart_fitness_value = rhc_best_fitness
        elif rhc_best_fitness > best_restart_fitness_value:
            best_restart_param = i
            best_restart_fitness_value = rhc_best_fitness
    print("Best RHC parameters for {} = {}".format(func[0], str(best_restart_param)))

Find best RHC hyperparam for <mlrose_hiive.fitness.four_peaks.FourPeaks object at 0x121766f40>
Best RHC parameters for fourpeaks = 100
Find best RHC hyperparam for <mlrose_hiive.fitness.continuous_peaks.ContinuousPeaks object at 0x121766be0>
Best RHC parameters for contpeaks = 100
Find best RHC hyperparam for <mlrose_hiive.fitness.one_max.OneMax object at 0x1217661f0>
Best RHC parameters for onemax = 100


In [55]:
# SA Tuning
import itertools

# init_temp, decay, min_temp
sa_hyperparams = [
   [1, 2, 4, 8, 16, 32, 64],
   [0.1, 0.2, 0.4, 0.8],
   [0.001, 0.01, 0.1, 1]
]

best_param = None
best_fitness_value = None
for func in fitness_functions:
    fitness_function = func[1]
    print("Find best SA hyperparam for {}".format(str(fitness_function)))
    for i in itertools.product(*sa_hyperparams):
        problem = mlrose_hiive.DiscreteOpt(length=100, fitness_fn=fitness_function, maximize=True)
        decay = mlrose_hiive.GeomDecay(init_temp = i[0], decay=i[1], min_temp=i[2])
        sa_best_state, sa_best_fitness, sa_fitness_curve = mlrose_hiive.simulated_annealing(
                                                    problem, 
                                                    max_attempts=500, 
                                                    max_iters=500, 
                                                    curve=True, 
                                                    random_state=42,
                                                    schedule=decay)
        if not best_fitness_value:
            best_param = i
            best_fitness_value = sa_best_fitness
        elif sa_best_fitness > best_fitness_value:
            best_param = i
            best_fitness_value = sa_best_fitness
    print("Best SA parameters for {} = {}".format(func[0], str(best_param)))

Find best SA hyperparam for <mlrose_hiive.fitness.four_peaks.FourPeaks object at 0x121766f40>
Best SA parameters for fourpeaks = (1, 0.1, 0.001)
Find best SA hyperparam for <mlrose_hiive.fitness.continuous_peaks.ContinuousPeaks object at 0x121766be0>
Best SA parameters for contpeaks = (1, 0.1, 1)
Find best SA hyperparam for <mlrose_hiive.fitness.one_max.OneMax object at 0x1217661f0>
Best SA parameters for onemax = (1, 0.1, 1)


In [59]:
# GA Tuning

import itertools

# init_temp, decay, min_temp
ga_hyperparams = [
   [100, 200, 400],
   [0.2, 0.4, 0.8]
]

best_param = None
best_fitness_value = None
for func in fitness_functions:
    fitness_function = func[1]
    print("Find best GA hyperparam for {}".format(str(fitness_function)))
    best_param = None
    best_fitness_value = None
    for i in itertools.product(*ga_hyperparams):
        problem = mlrose_hiive.DiscreteOpt(length=100, fitness_fn=fitness_function, maximize=True)
        ga_best_state, ga_best_fitness, ga_fitness_curve = mlrose_hiive.genetic_alg(
                                                    problem, 
                                                    max_attempts=500, 
                                                    max_iters=500, 
                                                    curve=True, 
                                                    random_state=42,
                                                    pop_size=i[0],
                                                    mutation_prob=i[1])
        if not best_fitness_value:
            best_param = i
            best_fitness_value = ga_best_fitness
        elif ga_best_fitness > best_fitness_value:
            best_param = i
            best_fitness_value = ga_best_fitness
    print("Best parameters for {} = {}".format(func[0], str(best_param)))

Find best GA hyperparam for <mlrose_hiive.fitness.four_peaks.FourPeaks object at 0x121766f40>
Best parameters for fourpeaks = (200, 0.2)
Find best GA hyperparam for <mlrose_hiive.fitness.continuous_peaks.ContinuousPeaks object at 0x121766be0>
Best parameters for contpeaks = (200, 0.8)
Find best GA hyperparam for <mlrose_hiive.fitness.one_max.OneMax object at 0x1217661f0>
Best parameters for onemax = (100, 0.2)


In [61]:
# MIMIC Tuning
best_param = None
best_fitness_value = None
for func in fitness_functions:
    fitness_function = func[1]
    print("Find best MIMIC hyperparam for {}".format(str(fitness_function)))
    best_param = None
    best_fitness_value = None
    for i in [0.25, 0.5, 0.75]:
        problem = mlrose_hiive.DiscreteOpt(length=100, fitness_fn=fitness_function, maximize=True)
        mimic_best_state, mimic_best_fitness, mimic_fitness_curve = mlrose_hiive.mimic(
                                                            problem, 
                                                            max_attempts = 100, 
                                                            max_iters = 100,  
                                                            curve = True, 
                                                            random_state = 42)
        if not best_fitness_value:
            best_param = i
            best_fitness_value = mimic_best_fitness
        elif mimic_best_fitness > best_fitness_value:
            best_param = i
            best_fitness_value = mimic_best_fitness
    print("Best parameters for {} = {}".format(func[0], str(best_param)))

Find best MIMIC hyperparam for <mlrose_hiive.fitness.four_peaks.FourPeaks object at 0x121766f40>
Best parameters for fourpeaks = 0.25
Find best MIMIC hyperparam for <mlrose_hiive.fitness.continuous_peaks.ContinuousPeaks object at 0x121766be0>
Best parameters for contpeaks = 0.25
Find best MIMIC hyperparam for <mlrose_hiive.fitness.one_max.OneMax object at 0x1217661f0>
Best parameters for onemax = 0.25
